El objetivo de este enfoque, sera generar un/os modelo/s para predecir los atributos del dia siguiente al ultimo disponible en el dataset. Aqui se aplicaran 2 enfoques:

- Un modelo que prediga todas las variablse en simultaneo (con el objetivo de captar la interrelacion entre las mismas).
- Un modelo que prediga solamente la variable target (incialmente se realizaran pruebas con la variable Close, y luego se procedera a usar la variable Tendencia).

Una vez realiza la prediccion de los atributos del dia siguiente, se procedera a realizar la prediccion de la Tendencia/Close, se realimientara el dataset, y se procedere a predecir otro dia, repitiendo esto N veces.

### Imports

In [24]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import accuracy_score
from pmdarima.arima import auto_arima
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from neuralprophet import NeuralProphet
from tensorflow.keras.layers import Reshape
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
import datetime
from prophet import Prophet
from prophet.diagnostics import performance_metrics
from sklearn.model_selection import GridSearchCV
from sklearn.base import RegressorMixin
from scikeras.wrappers import KerasRegressor
from sklearn.metrics import make_scorer
from keras.callbacks import Callback
from sklearn.base import clone
from sklearn.ensemble import VotingRegressor
from tensorflow.keras.regularizers import l2
from skopt import BayesSearchCV

pd.set_option('display.max_columns', None)

# LSTM Predictor

### Dataset

In [2]:
dataset = pd.read_csv('/Users/mmarchetta/Desktop/Tesis-2024/data-visualization/final_dataset.csv')

dataset['Open_time'] = pd.to_datetime(dataset['Open_time'])
dates = dataset['Open_time'][:-5]

dataset.drop(['Sentimiento'], axis=1, inplace=True)
dataset.drop(['Sentimiento_coin'], axis=1, inplace=True)
dataset.drop(['Sentimiento_referentes'], axis=1, inplace=True)
dataset.drop(columns=['Open_time'], inplace=True)

dataset = dataset.round(2) # Limitar los valores float a 2 decimales en todo el dataframe

feature_dataset = dataset
feature_dataset.drop(['Tendencia'], axis=1, inplace=True)

validation = feature_dataset[-5:]
feature_dataset = feature_dataset[:-5]

n_days_to_predict = 5

In [3]:
display(feature_dataset.head())
print(feature_dataset.shape)

display(validation.head())
display(validation.shape)

,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Open_BTCUSDT,High_BTCUSDT,Low_BTCUSDT,Close_BTCUSDT,Volume_BTCUSDT,Quote_asset_volume_BTCUSDT,Number_of_trades_BTCUSDT,Open_ETHUSDT,High_ETHUSDT,Low_ETHUSDT,Close_ETHUSDT,Volume_ETHUSDT,Quote_asset_volume_ETHUSDT,Number_of_trades_ETHUSDT,Open_BNBUSDT,High_BNBUSDT,Low_BNBUSDT,Close_BNBUSDT,Volume_BNBUSDT,Quote_asset_volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,MACD,Signal,ADX,SlowK,SlowD,CCI,ATR,MFI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Cantidad_pos,Cantidad_neg,Cantidad_neu,Compound total,Tweets_Utilizados_coin,Cantidad_pos_coin,Cantidad_neg_coin,Cantidad_neu_coin,Compound total coin,Tweets_Utilizados_referentes,Cantidad_pos_referentes,Cantidad_neg_referentes,Cantidad_neu_referentes,Compound total referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance
0,28.84,30.26,27.50,27.71,11094418.10,3.203400e+08,449178.0,43160.90,44350.00,42098.00,42147.35,39776.84,1.728912e+09,1001487.0,3059.36,3164.94,2922.71,2925.59,510130.73,1.564097e+09,1043885.0,344.0,357.6,335.0,335.5,956544.07,3.323725e+08,457187.0,31.85,30.83,38.30,31.85,25.39,44.11,0.02,0.96,21.20,24.79,33.78,-86.30,4.00,29.23,0.0,0.0,0.0,139,61.0,28.0,50.0,17.83,135,71.0,26.0,38.0,30.62,1.0,0.0,0.0,1.0,0.00,81.0,525.0,364.0,270000.0
1,27.72,28.38,26.14,26.31,9784154.53,2.675610e+08,362304.0,42147.35,42787.38,40888.00,41026.54,43372.26,1.816305e+09,1045389.0,2924.76,2970.82,2789.00,2804.91,511325.46,1.477983e+09,928494.0,335.5,344.6,330.0,333.0,922077.23,3.108283e+08,417006.0,31.77,30.40,38.44,31.77,25.11,41.83,-0.33,0.70,21.98,16.10,24.47,-109.46,3.88,19.19,4.0,2.0,5273.0,93,40.0,21.0,32.0,10.18,122,57.0,23.0,42.0,19.89,2.0,1.0,1.0,0.0,0.02,87.0,472.0,331.0,204000.0
2,26.31,28.59,26.11,27.28,10207287.33,2.805354e+08,376232.0,41025.01,42590.00,40753.88,41524.28,33511.53,1.399625e+09,884909.0,2805.15,2950.00,2782.00,2850.45,411305.09,1.180681e+09,748804.0,332.9,375.4,331.2,367.7,1696420.04,6.125477e+08,653011.0,31.65,30.10,38.55,31.65,24.74,43.99,-0.52,0.46,22.61,12.88,17.92,-92.93,3.78,23.82,22.0,40.0,54144.0,112,61.0,19.0,32.0,23.13,145,71.0,20.0,54.0,30.30,0.0,0.0,0.0,0.0,0.00,64.0,594.0,495.0,216000.0
3,27.28,28.99,27.13,28.62,8201551.44,2.306230e+08,339737.0,41524.29,44141.37,41410.17,43824.10,46381.23,2.005486e+09,1197815.0,2850.45,3048.42,2837.19,3000.61,506896.76,1.514342e+09,992243.0,367.7,388.8,366.4,387.5,1163674.21,4.396286e+08,551245.0,31.62,29.96,38.56,31.62,24.67,46.92,-0.56,0.26,22.99,16.42,15.13,-60.03,3.64,28.34,15.0,23.0,39220.0,116,46.0,28.0,42.0,7.68,147,73.0,26.0,48.0,30.90,2.0,1.0,0.0,1.0,0.75,77.0,419.0,464.0,202000.0
4,28.61,32.33,28.50,31.94,13176348.21,4.027010e+08,735059.0,43820.01,48495.00,43283.03,48141.61,66244.87,3.076095e+09,1771237.0,3000.62,3330.00,2969.07,3309.91,648714.62,2.063112e+09,1446386.0,387.5,423.4,381.5,421.5,1440336.04,5.843112e+08,727854.0,31.64,30.15,38.58,31.64,24.69,53.42,-0.31,0.14,21.86,34.90,21.40,34.17,3.65,35.02,24.0,35.0,63183.0,171,72.0,21.0,78.0,30.72,141,70.0,18.0,53.0,30.43,1.0,1.0,0.0,0.0,0.82,71.0,477.0,664.0,492000.0


(903, 64)


,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Open_BTCUSDT,High_BTCUSDT,Low_BTCUSDT,Close_BTCUSDT,Volume_BTCUSDT,Quote_asset_volume_BTCUSDT,Number_of_trades_BTCUSDT,Open_ETHUSDT,High_ETHUSDT,Low_ETHUSDT,Close_ETHUSDT,Volume_ETHUSDT,Quote_asset_volume_ETHUSDT,Number_of_trades_ETHUSDT,Open_BNBUSDT,High_BNBUSDT,Low_BNBUSDT,Close_BNBUSDT,Volume_BNBUSDT,Quote_asset_volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,MACD,Signal,ADX,SlowK,SlowD,CCI,ATR,MFI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Cantidad_pos,Cantidad_neg,Cantidad_neu,Compound total,Tweets_Utilizados_coin,Cantidad_pos_coin,Cantidad_neg_coin,Cantidad_neu_coin,Compound total coin,Tweets_Utilizados_referentes,Cantidad_pos_referentes,Cantidad_neg_referentes,Cantidad_neu_referentes,Compound total referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance
903,10.08,10.46,9.60,9.90,9838197.40,9.868151e+07,245319.0,68393.47,68956.00,66565.20,67609.99,55691.08,3.768278e+09,2464515.0,3644.70,3645.02,3454.09,3520.46,570901.29,2.029691e+09,1906387.0,571.7,587.7,540.1,555.4,2284301.81,1.280062e+09,994512.0,10.06,9.95,11.86,10.06,8.26,52.48,0.58,0.73,20.17,46.22,56.05,-67.98,0.95,48.31,34.0,43.0,84706.0,696,372.0,133.0,191.0,144.56,471,239.0,84.0,148.0,100.92,0.0,0.0,0.0,0.0,0.00,43.0,343.0,228.0,154000.0
904,9.90,9.99,8.60,8.77,15363075.80,1.402230e+08,341363.0,67610.00,68124.11,61555.00,61937.40,101005.32,6.484806e+09,3593832.0,3520.47,3548.09,3150.88,3158.64,1049629.69,3.474308e+09,2647385.0,555.4,560.6,495.8,507.7,2551361.51,1.331115e+09,1213572.0,10.08,9.84,11.81,10.08,8.35,42.93,0.42,0.67,20.01,34.35,44.98,-190.87,0.99,52.24,120.0,126.0,135180.0,961,481.0,185.0,295.0,177.84,509,283.0,90.0,136.0,133.03,1.0,1.0,0.0,0.0,0.44,56.0,534.0,433.0,221000.0
905,8.77,9.57,8.49,9.48,12729562.23,1.151930e+08,267797.0,61937.41,68100.00,60775.00,67840.51,90420.59,5.769770e+09,3549793.0,3158.65,3535.24,3056.56,3516.53,1207322.82,3.957251e+09,2987953.0,507.7,559.0,500.0,556.8,1425296.58,7.491883e+08,809335.0,10.14,9.80,11.68,10.14,8.60,49.21,0.34,0.60,19.98,26.96,35.84,-150.24,0.99,50.86,185.0,117.0,112997.0,866,463.0,147.0,256.0,199.18,555,292.0,94.0,169.0,130.81,1.0,1.0,0.0,0.0,0.44,40.0,473.0,386.0,171000.0
906,9.48,9.58,9.07,9.18,7536093.12,7.032619e+07,156774.0,67840.51,68240.47,64529.01,65501.27,53357.48,3.551223e+09,2388390.0,3516.53,3587.32,3412.00,3492.85,602755.21,2.114085e+09,1791989.0,556.7,571.0,540.6,553.8,953921.37,5.304850e+08,563996.0,10.17,9.74,11.63,10.17,8.71,46.85,0.26,0.53,19.93,18.21,26.51,-111.92,0.96,49.35,64.0,81.0,66543.0,692,396.0,103.0,193.0,183.34,533,284.0,86.0,163.0,139.78,0.0,0.0,0.0,0.0,0.00,24.0,350.0,290.0,101000.0
907,9.18,9.37,8.69,8.94,6104700.92,5.501917e+07,147578.0,65501.28,66649.62,62260.00,63796.64,51482.38,3.310482e+09,2492881.0,3492.84,3542.52,3250.00,3336.35,558848.89,1.891772e+09,1747756.0,553.8,588.1,535.5,553.8,1181298.51,6.651462e+08,712381.0,10.14,9.67,11.67,10.14,8.62,45.00,0.16,0.46,20.33,20.92,22.03,-116.79,0.94,51.61,57.0,66.0,68616.0,681,353.0,95.0,233.0,150.80,546,283.0,112.0,151.0,108.96,0.0,0.0,0.0,0.0,0.00,41.0,252.0,206.0,92000.0


(5, 64)

### Scalers

In [4]:
scalers = {}
for col in feature_dataset.columns:
    scaler = MinMaxScaler(feature_range=(0, 1))
    feature_dataset[col] = scaler.fit_transform(np.array(feature_dataset[col]).reshape(-1, 1))
    scalers[col] = scaler


WARNING - (py.warnings._showwarnmsg) - /var/folders/ls/f33jscqd6sb6bd857yb2j2s00000gp/T/ipykernel_19572/3132932540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_dataset[col] = scaler.fit_transform(np.array(feature_dataset[col]).reshape(-1, 1))

WARNING - (py.warnings._showwarnmsg) - /var/folders/ls/f33jscqd6sb6bd857yb2j2s00000gp/T/ipykernel_19572/3132932540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_dataset[col] = scaler.fit_transform(np.array(feature_dataset[col]).reshape(-1,

In [5]:
display(feature_dataset.head())
print(feature_dataset.shape)

display(validation.head())
display(validation.shape)

,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Open_BTCUSDT,High_BTCUSDT,Low_BTCUSDT,Close_BTCUSDT,Volume_BTCUSDT,Quote_asset_volume_BTCUSDT,Number_of_trades_BTCUSDT,Open_ETHUSDT,High_ETHUSDT,Low_ETHUSDT,Close_ETHUSDT,Volume_ETHUSDT,Quote_asset_volume_ETHUSDT,Number_of_trades_ETHUSDT,Open_BNBUSDT,High_BNBUSDT,Low_BNBUSDT,Close_BNBUSDT,Volume_BNBUSDT,Quote_asset_volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,MACD,Signal,ADX,SlowK,SlowD,CCI,ATR,MFI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Cantidad_pos,Cantidad_neg,Cantidad_neu,Compound total,Tweets_Utilizados_coin,Cantidad_pos_coin,Cantidad_neg_coin,Cantidad_neu_coin,Compound total coin,Tweets_Utilizados_referentes,Cantidad_pos_referentes,Cantidad_neg_referentes,Cantidad_neu_referentes,Compound total referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance
0,0.501993,0.517208,0.492593,0.479569,0.290506,0.223416,0.168108,0.477903,0.487888,0.476607,0.460212,0.040979,0.085860,0.046312,0.541389,0.550539,0.532613,0.506304,0.126579,0.296767,0.285966,0.321945,0.325617,0.336067,0.303395,0.197596,0.175175,0.238897,0.633771,0.615034,0.648008,0.633771,0.602961,0.345342,0.503145,0.634483,0.288525,0.223198,0.320831,0.391348,0.977330,0.266878,0.000000,0.000000,0.000000,0.123457,0.169916,0.142132,0.148734,0.091285,0.136662,0.118881,0.190909,0.144144,0.133100,0.058824,0.00,0.00,0.166667,0.197248,0.536424,0.302018,0.212508,0.140182
1,0.479673,0.480653,0.464609,0.451664,0.253904,0.186206,0.134757,0.460212,0.460694,0.454945,0.440649,0.045762,0.090937,0.049256,0.506087,0.499309,0.497723,0.474653,0.126913,0.279364,0.248880,0.303329,0.297490,0.324983,0.297919,0.189790,0.163080,0.215055,0.631960,0.605239,0.650664,0.631960,0.595284,0.307138,0.459119,0.598621,0.305771,0.131231,0.215657,0.353485,0.947103,0.153149,0.016878,0.007092,0.026035,0.079772,0.111421,0.106599,0.091772,0.063540,0.119580,0.086247,0.163636,0.162162,0.077072,0.117647,0.25,0.25,0.000000,0.199541,0.576159,0.268600,0.191449,0.104869
2,0.451574,0.484737,0.463992,0.470999,0.265724,0.195353,0.140104,0.440622,0.457259,0.452544,0.449336,0.032645,0.066732,0.038494,0.474717,0.493814,0.495897,0.486597,0.098880,0.219282,0.191129,0.297635,0.364128,0.327644,0.373932,0.365156,0.332474,0.355091,0.629244,0.598405,0.652751,0.629244,0.585138,0.343331,0.435220,0.565517,0.319699,0.097153,0.141663,0.380509,0.921914,0.205596,0.092827,0.141844,0.267329,0.097816,0.169916,0.096447,0.091772,0.110507,0.149803,0.118881,0.136364,0.216216,0.131429,0.000000,0.00,0.00,0.000000,0.197248,0.423841,0.345523,0.296107,0.111289
3,0.470905,0.492514,0.484979,0.497708,0.209695,0.160164,0.126093,0.449337,0.484257,0.464293,0.489479,0.049765,0.101927,0.059477,0.486598,0.519788,0.510298,0.525979,0.125672,0.286712,0.269369,0.373850,0.393120,0.405675,0.417306,0.244504,0.235392,0.294707,0.628565,0.595216,0.652941,0.628565,0.583219,0.392426,0.430189,0.537931,0.328101,0.134617,0.110145,0.434296,0.886650,0.256797,0.063291,0.081560,0.193644,0.101614,0.128134,0.142132,0.123418,0.054474,0.152431,0.123543,0.190909,0.189189,0.134562,0.117647,0.25,0.00,0.166667,0.283257,0.509934,0.235183,0.276324,0.103799
4,0.497409,0.557457,0.513169,0.563883,0.348664,0.281482,0.277858,0.489408,0.560022,0.497823,0.564840,0.076188,0.164119,0.097929,0.525983,0.594101,0.544710,0.607100,0.165421,0.397615,0.415326,0.417214,0.467979,0.439149,0.491785,0.307160,0.316621,0.399500,0.629018,0.599544,0.653321,0.629018,0.583767,0.501340,0.461635,0.521379,0.303117,0.330194,0.180976,0.588298,0.889169,0.332465,0.101266,0.124113,0.311958,0.153846,0.200557,0.106599,0.237342,0.138034,0.144547,0.116550,0.118182,0.211712,0.132108,0.058824,0.25,0.00,0.000000,0.291284,0.470199,0.271753,0.403957,0.258962


(903, 64)


,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Open_BTCUSDT,High_BTCUSDT,Low_BTCUSDT,Close_BTCUSDT,Volume_BTCUSDT,Quote_asset_volume_BTCUSDT,Number_of_trades_BTCUSDT,Open_ETHUSDT,High_ETHUSDT,Low_ETHUSDT,Close_ETHUSDT,Volume_ETHUSDT,Quote_asset_volume_ETHUSDT,Number_of_trades_ETHUSDT,Open_BNBUSDT,High_BNBUSDT,Low_BNBUSDT,Close_BNBUSDT,Volume_BNBUSDT,Quote_asset_volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,MACD,Signal,ADX,SlowK,SlowD,CCI,ATR,MFI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Cantidad_pos,Cantidad_neg,Cantidad_neu,Compound total,Tweets_Utilizados_coin,Cantidad_pos_coin,Cantidad_neg_coin,Cantidad_neu_coin,Compound total coin,Tweets_Utilizados_referentes,Cantidad_pos_referentes,Cantidad_neg_referentes,Cantidad_neu_referentes,Compound total referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance
903,10.08,10.46,9.60,9.90,9838197.40,9.868151e+07,245319.0,68393.47,68956.00,66565.20,67609.99,55691.08,3.768278e+09,2464515.0,3644.70,3645.02,3454.09,3520.46,570901.29,2.029691e+09,1906387.0,571.7,587.7,540.1,555.4,2284301.81,1.280062e+09,994512.0,10.06,9.95,11.86,10.06,8.26,52.48,0.58,0.73,20.17,46.22,56.05,-67.98,0.95,48.31,34.0,43.0,84706.0,696,372.0,133.0,191.0,144.56,471,239.0,84.0,148.0,100.92,0.0,0.0,0.0,0.0,0.00,43.0,343.0,228.0,154000.0
904,9.90,9.99,8.60,8.77,15363075.80,1.402230e+08,341363.0,67610.00,68124.11,61555.00,61937.40,101005.32,6.484806e+09,3593832.0,3520.47,3548.09,3150.88,3158.64,1049629.69,3.474308e+09,2647385.0,555.4,560.6,495.8,507.7,2551361.51,1.331115e+09,1213572.0,10.08,9.84,11.81,10.08,8.35,42.93,0.42,0.67,20.01,34.35,44.98,-190.87,0.99,52.24,120.0,126.0,135180.0,961,481.0,185.0,295.0,177.84,509,283.0,90.0,136.0,133.03,1.0,1.0,0.0,0.0,0.44,56.0,534.0,433.0,221000.0
905,8.77,9.57,8.49,9.48,12729562.23,1.151930e+08,267797.0,61937.41,68100.00,60775.00,67840.51,90420.59,5.769770e+09,3549793.0,3158.65,3535.24,3056.56,3516.53,1207322.82,3.957251e+09,2987953.0,507.7,559.0,500.0,556.8,1425296.58,7.491883e+08,809335.0,10.14,9.80,11.68,10.14,8.60,49.21,0.34,0.60,19.98,26.96,35.84,-150.24,0.99,50.86,185.0,117.0,112997.0,866,463.0,147.0,256.0,199.18,555,292.0,94.0,169.0,130.81,1.0,1.0,0.0,0.0,0.44,40.0,473.0,386.0,171000.0
906,9.48,9.58,9.07,9.18,7536093.12,7.032619e+07,156774.0,67840.51,68240.47,64529.01,65501.27,53357.48,3.551223e+09,2388390.0,3516.53,3587.32,3412.00,3492.85,602755.21,2.114085e+09,1791989.0,556.7,571.0,540.6,553.8,953921.37,5.304850e+08,563996.0,10.17,9.74,11.63,10.17,8.71,46.85,0.26,0.53,19.93,18.21,26.51,-111.92,0.96,49.35,64.0,81.0,66543.0,692,396.0,103.0,193.0,183.34,533,284.0,86.0,163.0,139.78,0.0,0.0,0.0,0.0,0.00,24.0,350.0,290.0,101000.0
907,9.18,9.37,8.69,8.94,6104700.92,5.501917e+07,147578.0,65501.28,66649.62,62260.00,63796.64,51482.38,3.310482e+09,2492881.0,3492.84,3542.52,3250.00,3336.35,558848.89,1.891772e+09,1747756.0,553.8,588.1,535.5,553.8,1181298.51,6.651462e+08,712381.0,10.14,9.67,11.67,10.14,8.62,45.00,0.16,0.46,20.33,20.92,22.03,-116.79,0.94,51.61,57.0,66.0,68616.0,681,353.0,95.0,233.0,150.80,546,283.0,112.0,151.0,108.96,0.0,0.0,0.0,0.0,0.00,41.0,252.0,206.0,92000.0


(5, 64)

### Preparo el dataset para train: cada conjunto de entrenamiento, sera una seried de N dias previos, para predecir 1 dia siguiente.

In [6]:
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        end_ix = i + n_steps
        seq_x = data.iloc[i:end_ix, :].values
        seq_y = data.iloc[end_ix, :].values
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)


n_steps = 30  # Longitud de la secuencia de entrada
n_features = feature_dataset.shape[1]  # Número de características

# Crear las secuencias de entrada y salida
X, y = create_sequences(feature_dataset, n_steps)

In [7]:
print(X[0].shape) # Cada dato de entrenamiento, es un conjunto de 30 dias con sus 64 features
print(y[0].shape) # El target de cada dato, son los 64 features del dia siguiente
print(X.shape)
print(y.shape)
print(feature_dataset.shape)

(30, 64)
(64,)
(873, 30, 64)
(873, 64)
(903, 64)


In [8]:
print(len(X))
print(len(y))

873
873


### Obtencion de los mejores hiperparametros

In [9]:
def custom_scoring(estimator, X, y):
    y_pred = estimator.predict(X)
    mse = mean_squared_error(y, y_pred)
    return -mse

In [10]:
def custom_scoring_validation(y, y_pred):
    mse = mean_squared_error(y, y_pred)
    return -mse

In [130]:
# X_reshaped = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))
# print(X_reshaped.shape)


In [37]:
import warnings
warnings.filterwarnings('ignore')

def create_model(activation, units, dropout, learning_rate, l2_penalty):
    model = Sequential([
        LSTM(units=units, activation=activation, input_shape=(n_steps, n_features), return_sequences=True, kernel_regularizer=l2(l2_penalty)),
        Dropout(dropout),
        LSTM(units=units, activation=activation, return_sequences=True, kernel_regularizer=l2(l2_penalty)),
        Dropout(dropout),
        LSTM(units=units, activation=activation, kernel_regularizer=l2(l2_penalty)),
        Dropout(dropout),
        Dense(units=n_features)
    ])
    
    # if optimizer == 'adam':
    optimizer = Adam(learning_rate=learning_rate)
    # elif optimizer == 'rmsprop':
    #     optimizer = RMSprop(learning_rate=learning_rate)
    # elif optimizer == 'sgd':
    #     optimizer = SGD(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='mse', metrics=['accuracy'])
    return model

regressor = KerasRegressor(build_fn=create_model, verbose=0, activation='relu', units=50, dropout=0.2, learning_rate=0.1, l2_penalty=0.001)

cv = TimeSeriesSplit(n_splits=20).split(X)
param_space = {
    'activation': ['relu', 'tanh'],
    'units': [50, 100, 150],
    'dropout': [0.2, 0.3, 0.4],
    'learning_rate': [0.01, 0.001, 0.0001],
    'epochs': [10, 20, 30],
    'batch_size': [32, 64, 128],
    # 'optimizer': ['adam', 'rmsprop', 'sgd'], <= por algun motivo tira error
    'l2_penalty': [0.001, 0.01, 0.1]
}

bayes_search = BayesSearchCV(regressor, param_space, scoring=custom_scoring, cv=cv, verbose=10)
bayes_result = bayes_search.fit(X, y)

[CV 15/20; 1/1] END activation=tanh, batch_size=128, dropout=0.2, epochs=30, l2_penalty=0.1, learning_rate=0.001, units=150;, score=-0.012 total time= 1.5min
[CV 16/20; 1/1] START activation=tanh, batch_size=128, dropout=0.2, epochs=30, l2_penalty=0.1, learning_rate=0.001, units=150
[CV 16/20; 1/1] END activation=tanh, batch_size=128, dropout=0.2, epochs=30, l2_penalty=0.1, learning_rate=0.001, units=150;, score=-0.010 total time= 1.8min
[CV 17/20; 1/1] START activation=tanh, batch_size=128, dropout=0.2, epochs=30, l2_penalty=0.1, learning_rate=0.001, units=150
[CV 17/20; 1/1] END activation=tanh, batch_size=128, dropout=0.2, epochs=30, l2_penalty=0.1, learning_rate=0.001, units=150;, score=-0.022 total time= 1.7min
[CV 18/20; 1/1] START activation=tanh, batch_size=128, dropout=0.2, epochs=30, l2_penalty=0.1, learning_rate=0.001, units=150
[CV 18/20; 1/1] END activation=tanh, batch_size=128, dropout=0.2, epochs=30, l2_penalty=0.1, learning_rate=0.001, units=150;, score=-0.030 total tim

In [39]:
# Show best results
print("Best score:", bayes_result.best_score_)
print("Best parameters:", bayes_result.best_params_)

# Entrenar el modelo con los mejores hiperparámetros
best_model = bayes_result.best_estimator_
best_model.fit(X, y)

Best score: -0.015204753533642767
Best parameters: OrderedDict([('activation', 'tanh'), ('batch_size', 64), ('dropout', 0.3), ('epochs', 10), ('l2_penalty', 0.001), ('learning_rate', 0.001), ('units', 150)])


KerasRegressor(
	model=None
	build_fn=<function create_model at 0x7f83797dc040>
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=64
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=10
	activation=tanh
	units=150
	dropout=0.3
	learning_rate=0.001
	l2_penalty=0.001
)

### Predicciones con el mejor conjunto de hiper parametros

In [47]:
n_days_to_predict = 5
future_dataset = feature_dataset

dataset = pd.read_csv('/Users/mmarchetta/Desktop/Tesis-2024/data-visualization/final_dataset.csv')
dataset['Open_time'] = pd.to_datetime(dataset['Open_time'])
dates = dataset['Open_time']

# Crear un DataFrame vacío para almacenar las predicciones desnormalizadas
predicted_values_desnormalized = pd.DataFrame(columns=future_dataset.columns)

# Lista para almacenar las fechas de las predicciones
predicted_dates = []

for _ in range(n_days_to_predict):
    # Predecir 1 día posterior al último día disponible en el dataset
    last_sequence = future_dataset.iloc[-n_steps:, :].values.reshape((1, n_steps, n_features))
    predictions = best_model.predict(last_sequence)

    # Agregar las predicciones sin desnormalizar a future_dataset
    predicted_values_normalized = pd.DataFrame(predictions, columns=future_dataset.columns)
    future_dataset = pd.concat([future_dataset, predicted_values_normalized], axis=0, ignore_index=True)

    # Desnormalizar las predicciones y agregarlas al DataFrame de predicciones desnormalizadas
    inverted_predictions = []
    for i in range(len(future_dataset.columns)):
        col = future_dataset.columns[i]
        scaler = scalers[col]
        prediction = predictions[:, i].reshape(-1, 1)
        inverted_prediction = scaler.inverse_transform(prediction)
        inverted_predictions.append(inverted_prediction)

    # Calcular la fecha del próximo día
    next_day_date = dates.iloc[-1] + pd.DateOffset(days=1)
    predicted_dates.append(next_day_date)

    # Actualizar la fecha del próximo día en el DataFrame principal
    dates = dates.append(pd.Series([next_day_date], name='Fecha'))

    # Crear un DataFrame con las predicciones desnormalizadas
    predicted_values_desnormalized = pd.concat([predicted_values_desnormalized,
                                                pd.DataFrame(np.concatenate(inverted_predictions, axis=1),
                                                             columns=future_dataset.columns)], 
                                                ignore_index=True)

# Agregar las fechas al DataFrame de predicciones desnormalizadas
predicted_values_desnormalized['Fecha'] = predicted_dates

print("Valores predichos para los próximos {} días:".format(n_days_to_predict))
display(future_dataset.tail(n_days_to_predict + 1))

print("Valores predichos desnormalizados para los próximos {} días:".format(n_days_to_predict))
display(predicted_values_desnormalized.tail(n_days_to_predict))


Valores predichos para los próximos 5 días:


,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Open_BTCUSDT,High_BTCUSDT,Low_BTCUSDT,Close_BTCUSDT,Volume_BTCUSDT,Quote_asset_volume_BTCUSDT,Number_of_trades_BTCUSDT,Open_ETHUSDT,High_ETHUSDT,Low_ETHUSDT,Close_ETHUSDT,Volume_ETHUSDT,Quote_asset_volume_ETHUSDT,Number_of_trades_ETHUSDT,Open_BNBUSDT,High_BNBUSDT,Low_BNBUSDT,Close_BNBUSDT,Volume_BNBUSDT,Quote_asset_volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,MACD,Signal,ADX,SlowK,SlowD,CCI,ATR,MFI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Cantidad_pos,Cantidad_neg,Cantidad_neu,Compound total,Tweets_Utilizados_coin,Cantidad_pos_coin,Cantidad_neg_coin,Cantidad_neu_coin,Compound total coin,Tweets_Utilizados_referentes,Cantidad_pos_referentes,Cantidad_neg_referentes,Cantidad_neu_referentes,Compound total referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance
902,0.120167,0.128135,0.115844,0.128164,0.243737,0.062840,0.083843,0.864346,0.915203,0.878256,0.918331,0.054236,0.178460,0.143388,0.663011,0.686122,0.660288,0.694908,0.128726,0.353230,0.503699,0.830924,0.829511,0.809577,0.820811,0.368893,0.541466,0.443672,0.138751,0.139408,0.147628,0.138751,0.127228,0.515583,0.584906,0.606897,0.298696,0.536141,0.724243,0.388226,0.211587,0.510988,0.151899,0.170213,0.352479,0.649573,1.000000,0.670051,0.629747,0.510644,0.501971,0.412587,0.618182,0.617117,0.429586,0.000000,0.000000,0.000000,0.000000,0.197248,0.139073,0.156999,0.156988,0.075976
903,0.357896,0.377132,0.320732,0.367270,0.348166,0.180514,0.155942,0.827393,0.832957,0.776071,0.810495,-0.031472,0.093408,0.061414,0.760217,0.776966,0.804451,0.787489,0.141186,0.347644,0.368239,0.609827,0.615954,0.631635,0.613217,0.317219,0.386690,0.372721,0.376817,0.373903,0.380321,0.351786,0.351660,0.713562,0.676053,0.666844,0.455376,0.714826,0.809812,0.677146,0.362090,0.750955,0.270108,0.214066,0.366064,0.203202,0.191248,0.231957,0.248970,0.171818,0.221005,0.204221,0.249556,0.244603,0.218165,0.054198,0.073967,0.044787,0.008304,0.246499,0.350475,0.286206,0.272475,0.099979
904,0.366029,0.385984,0.328160,0.375401,0.352222,0.184248,0.159050,0.840056,0.845737,0.786819,0.822235,-0.035372,0.092753,0.059527,0.772805,0.790033,0.816477,0.800840,0.141851,0.352101,0.372324,0.621199,0.627083,0.643216,0.624028,0.321927,0.393616,0.378179,0.386101,0.382840,0.390259,0.360521,0.360809,0.717041,0.678488,0.669216,0.455967,0.716448,0.814168,0.678709,0.370516,0.753332,0.271617,0.214882,0.371809,0.205515,0.195462,0.235145,0.251251,0.173749,0.222622,0.206022,0.251683,0.246970,0.220267,0.053092,0.075544,0.044414,0.007557,0.246270,0.356069,0.287656,0.274613,0.101856
905,0.374075,0.394617,0.335588,0.383353,0.355442,0.187686,0.161977,0.851018,0.856889,0.796075,0.832474,-0.039130,0.091943,0.057474,0.784301,0.801964,0.827211,0.812996,0.142172,0.355982,0.375569,0.631940,0.637444,0.654084,0.634148,0.325921,0.399744,0.383131,0.395523,0.391797,0.400275,0.369454,0.369986,0.718521,0.679460,0.670082,0.456026,0.716198,0.816303,0.678810,0.378992,0.753686,0.272304,0.214923,0.376957,0.207274,0.199275,0.237789,0.253002,0.175250,0.223818,0.207403,0.253286,0.248713,0.221921,0.051986,0.077028,0.043923,0.006828,0.245661,0.361284,0.288212,0.275969,0.103699
906,0.381295,0.402162,0.342196,0.390339,0.358166,0.190709,0.164742,0.862092,0.868243,0.805384,0.842888,-0.043872,0.090377,0.054341,0.795354,0.813330,0.837420,0.824734,0.142026,0.359189,0.378239,0.641471,0.646549,0.663852,0.643185,0.329919,0.405552,0.387485,0.403875,0.399687,0.409202,0.377305,0.378002,0.719668,0.679953,0.670521,0.455875,0.715663,0.818135,0.678342,0.386171,0.753635,0.273267,0.215154,0.381780,0.208965,0.202986,0.240438,0.254712,0.176871,0.224970,0.208623,0.254865,0.250220,0.223460,0.050417,0.078359,0.043129,0.005733,0.244607,0.366050,0.287962,0.276571,0.105325
907,0.388254,0.409334,0.348661,0.397024,0.360191,0.193425,0.167281,0.871522,0.878011,0.813300,0.851860,-0.048332,0.088773,0.051198,0.8

Valores predichos desnormalizados para los próximos 5 días:


,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Open_BTCUSDT,High_BTCUSDT,Low_BTCUSDT,Close_BTCUSDT,Volume_BTCUSDT,Quote_asset_volume_BTCUSDT,Number_of_trades_BTCUSDT,Open_ETHUSDT,High_ETHUSDT,Low_ETHUSDT,Close_ETHUSDT,Volume_ETHUSDT,Quote_asset_volume_ETHUSDT,Number_of_trades_ETHUSDT,Open_BNBUSDT,High_BNBUSDT,Low_BNBUSDT,Close_BNBUSDT,Volume_BNBUSDT,Quote_asset_volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,MACD,Signal,ADX,SlowK,SlowD,CCI,ATR,MFI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Cantidad_pos,Cantidad_neg,Cantidad_neu,Compound total,Tweets_Utilizados_coin,Cantidad_pos_coin,Cantidad_neg_coin,Cantidad_neu_coin,Compound total coin,Tweets_Utilizados_referentes,Cantidad_pos_referentes,Cantidad_neg_referentes,Cantidad_neu_referentes,Compound total referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance,Fecha
0,21.609215,23.055923,19.147562,22.075953,13158525.0,259487952.0,417487.21875,63183.558594,64178.402344,58825.257812,62215.441406,-14687.165039,1.858844e+09,1226697.750,3893.722412,4022.898193,3964.480957,3997.707031,562246.8750,1.815848e+09,1299875.000,475.446838,491.793976,468.330536,476.933411,1484751.500,709115584.0,682723.0625,20.497791,20.244350,24.192911,19.391901,16.225044,66.085388,1.394618,1.194619,28.746660,71.243927,77.064583,88.516388,1.557497,71.964310,64.015709,60.366726,74141.507812,222.971344,68.658020,45.695446,81.674492,40.035435,199.184555,107.610954,32.451126,60.301899,46.910721,0.921368,0.295867,0.179148,0.049826,0.429471,52.921757,499.922943,457.967712,194861.343750,2024-03-23
1,22.017342,23.511168,19.508581,22.483871,13303724.0,264783856.0,425584.31250,63909.031250,64912.714844,59425.574219,62888.039062,-17619.201172,1.847563e+09,1198561.125,3941.717041,4072.411133,4010.569824,4048.614258,564621.3750,1.837903e+09,1312585.500,480.639282,496.937866,473.554535,481.868622,1505539.250,721452480.0,691921.3750,20.907951,20.636677,24.716646,19.777826,16.558716,66.292984,1.413976,1.211818,28.773376,71.397194,77.450119,89.472633,1.590948,72.174156,64.373276,60.596611,75305.156250,225.407715,70.171005,46.323544,82.395279,40.567822,200.415329,108.383560,32.685169,60.827316,47.313335,0.902556,0.302175,0.177656,0.045341,0.427471,53.766411,502.222412,461.318878,198368.515625,2024-03-24
2,22.421082,23.955156,19.869564,22.882845,13418982.0,269660480.0,433208.09375,64537.070312,65553.578125,59942.636719,63474.671875,-20443.671875,1.833625e+09,1167941.875,3985.550293,4117.616211,4051.703613,4094.961914,565763.6250,1.857106e+09,1322682.250,485.543671,501.726532,478.457428,486.488770,1523174.750,732366016.0,700266.3125,21.324194,21.029869,25.244516,20.172489,16.893385,66.381302,1.421706,1.218093,28.776066,71.373573,77.639114,89.534653,1.624598,72.205444,64.536003,60.608147,76347.664062,227.259659,71.539848,46.844513,82.948746,40.981560,201.325745,108.975685,32.861458,61.214352,47.630116,0.883761,0.308112,0.175692,0.040965,0.422166,54.553955,503.104675,463.443451,201813.765625,2024-03-25
3,22.783407,24.343195,20.190704,23.233299,13516485.0,273948576.0,440410.18750,65171.507812,66206.000000,60462.585938,64071.292969,-24008.726562,1.806656e+09,1121218.375,4027.692383,4160.685547,4090.830811,4139.718750,565245.0000,1.872973e+09,1330991.250,489.895477,505.935120,482.863739,490.613892,1540830.750,742712000.0,707604.3125,21.693182,21.376244,25.714920,20.519327,17.185749,66.449806,1.425628,1.221276,28.769220,71.323006,77.801353,89.248123,1.653098,72.200943,64.764236,60.673397,77324.617188,229.040390,72.872070,47.366211,83.488991,41.428543,202.202194,109.499298,33.035137,61.548935,47.924889,0.857097,0.313437,0.172517,0.034400,0.412970,55.273502,502.708221,464.386627,204852.781250,2024-03-26
4,23.132582,24.712029,20.504919,23.568676,13588985.0,277800064.0,447025.21875,65711.781250,66767.265625,60904.730469,64585.285156,-27361.066406,1.779054e+09,1074348.375,4065.276855,4198.950684,412

### Guardado de los mejores hiperparametros

In [42]:
import json

# Obtener los hiperparámetros y puntajes de los 5 mejores modelos
top_n_models = 5
best_params_list = []
best_scores_list = []

for i in range(min(top_n_models, len(bayes_search.cv_results_['params']))):
    best_params_list.append(bayes_search.cv_results_['params'][i])
    best_scores_list.append(bayes_search.cv_results_['mean_test_score'][i])

# Guardar los hiperparámetros de los 5 mejores modelos en un archivo JSON
with open('top_5_hyperparameters.json', 'w') as f:
    json.dump({'best_params': best_params_list, 'best_scores': best_scores_list}, f)

# O imprimir los hiperparámetros
print("Top 5 mejores modelos:")
for i in range(len(best_params_list)):
    print("Modelo", i+1)
    print("Hiperparámetros:", best_params_list[i])
    print("Puntaje:", best_scores_list[i])


Top 5 mejores modelos:
Modelo 1
Hiperparámetros: OrderedDict([('activation', 'relu'), ('batch_size', 128), ('dropout', 0.2), ('epochs', 20), ('l2_penalty', 0.01), ('learning_rate', 0.0001), ('units', 150)])
Puntaje: -0.03986723249036641
Modelo 2
Hiperparámetros: OrderedDict([('activation', 'tanh'), ('batch_size', 128), ('dropout', 0.4), ('epochs', 30), ('l2_penalty', 0.01), ('learning_rate', 0.001), ('units', 150)])
Puntaje: -0.017320082783079418
Modelo 3
Hiperparámetros: OrderedDict([('activation', 'relu'), ('batch_size', 32), ('dropout', 0.4), ('epochs', 30), ('l2_penalty', 0.01), ('learning_rate', 0.001), ('units', 150)])
Puntaje: -0.018638353135904017
Modelo 4
Hiperparámetros: OrderedDict([('activation', 'relu'), ('batch_size', 64), ('dropout', 0.2), ('epochs', 10), ('l2_penalty', 0.001), ('learning_rate', 0.0001), ('units', 150)])
Puntaje: -0.038033967859191116
Modelo 5
Hiperparámetros: OrderedDict([('activation', 'relu'), ('batch_size', 32), ('dropout', 0.3), ('epochs', 10), ('l2

### Armado de un ensamble con los mejores 5 hiperparametros usando la mejor semilla en cada caso

In [43]:
def generate_prime_seeds(n):
    seeds = []
    num = 70001  # Comenzamos desde el primer número primo mayor que 70000
    while len(seeds) < n:
        is_prime = True
        for i in range(2, int(num**0.5) + 1):
            if num % i == 0:
                is_prime = False
                break
        if is_prime:
            seeds.append(num)
        num += 1
    return seeds


In [44]:
def predict_next_days(ensemble, feature_dataset, scalers, n_steps, n_features, n_days_to_predict):
    future_dataset = feature_dataset.copy()

    # Leer el conjunto de datos original para obtener las fechas
    dataset = pd.read_csv('/Users/mmarchetta/Desktop/Tesis-2024/data-visualization/final_dataset.csv')
    dataset['Open_time'] = pd.to_datetime(dataset['Open_time'])
    dates = dataset['Open_time'][:-n_days_to_predict]

    # Crear un DataFrame vacío para almacenar las predicciones desnormalizadas
    predicted_values_desnormalized = pd.DataFrame(columns=future_dataset.columns)

    # Lista para almacenar las fechas de las predicciones
    predicted_dates = []

    for _ in range(n_days_to_predict):
        # Predecir 1 día posterior al último día disponible en el dataset
        last_sequence = future_dataset.iloc[-n_steps:, :].values.reshape((1, n_steps, n_features))
        predictions = ensemble.predict(last_sequence)

        # Agregar las predicciones sin desnormalizar a future_dataset
        predicted_values_normalized = pd.DataFrame(predictions, columns=future_dataset.columns)
        future_dataset = pd.concat([future_dataset, predicted_values_normalized], axis=0, ignore_index=True)

        # Desnormalizar las predicciones y agregarlas al DataFrame de predicciones desnormalizadas
        inverted_predictions = []
        for i in range(len(future_dataset.columns)):
            col = future_dataset.columns[i]
            scaler = scalers[col]
            prediction = predictions[:, i].reshape(-1, 1)
            inverted_prediction = scaler.inverse_transform(prediction)
            inverted_predictions.append(inverted_prediction)

        # Calcular la fecha del próximo día
        next_day_date = dates.iloc[-1] + pd.DateOffset(days=1)
        predicted_dates.append(next_day_date)

        # Actualizar la fecha del próximo día en el DataFrame principal
        dates = dates.append(pd.Series([next_day_date], name='Fecha'))

        # Crear un DataFrame con las predicciones desnormalizadas
        predicted_values_desnormalized = pd.concat([predicted_values_desnormalized,
                                                    pd.DataFrame(np.concatenate(inverted_predictions, axis=1),
                                                                 columns=future_dataset.columns)], 
                                                    ignore_index=True)

    # Agregar las fechas al DataFrame de predicciones desnormalizadas
    predicted_values_desnormalized['Fecha'] = predicted_dates

    return future_dataset, predicted_values_desnormalized

In [45]:
## Clase personalizada para hacer el ensamble, dado que sklearn no provee ninguna clase que permita hacer ensmble
## de modelos re regresion multivariados
class MultivariableVotingRegressor:
    def __init__(self, models):
        self.models = models

    def fit(self, X, y):
        for model in self.models:
            model.fit(X, y)

    def predict(self, X):
        # Hacer predicciones con cada modelo
        predictions = [model.predict(X) for model in self.models]
    
        # Calcular el promedio de las predicciones
        average_predictions = np.mean(predictions, axis=0)
    
        return average_predictions


In [ ]:

import json

# Leer los hiperparámetros desde el archivo JSON
with open('top_5_hyperparameters.json', 'r') as f:
    top_hyperparameters = json.load(f)


models = []
best_seeds= {}
prime_seeds = generate_prime_seeds(300)

for mode_number, params in enumerate(top_hyperparameters['best_params']):
    best_validation_errors = {}
    
    for seend_number, seed in enumerate(prime_seeds):
        model = KerasRegressor(build_fn=create_model, random_state=seed, **params)
        
        model.fit(X, y)
        
        model_predictions, _ = predict_next_days(model, feature_dataset, scalers, n_steps, n_features, 5)

        error = custom_scoring_validation(validation, model_predictions[-5:])
        print(f"model number: {mode_number}, seed number: {seend_number} error: {error}")
        
        if seed not in best_validation_errors or error < best_validation_errors[seed]:
            best_validation_errors[seed] = error
    
    best_seed_for_params = min(best_validation_errors, key=best_validation_errors.get)
    best_seeds[str(params)] = best_seed_for_params
    
    model = KerasRegressor(build_fn=create_model, random_state=best_seed_for_params, **params)
    model.fit(X, y)
    models.append(model)


ensemble = MultivariableVotingRegressor(models)
ensemble.fit(X, y)

with open('best_seeds.json', 'w') as f:
    json.dump(best_seeds, f)

In [18]:
future_dataset = feature_dataset

future_dataset, predicted_values_desnormalized = predict_next_days(ensemble, feature_dataset, scalers, n_steps, n_features, n_days_to_predict)

print("Valores predichos para los próximos {} días:".format(n_days_to_predict))
display(future_dataset.tail(n_days_to_predict + 1))

print("Valores predichos desnormalizados para los próximos {} días:".format(n_days_to_predict))
display(predicted_values_desnormalized.tail(n_days_to_predict))


1/1 [==============================] - 0s 32ms/step
Valores predichos para los próximos 5 días:


,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Open_BTCUSDT,High_BTCUSDT,Low_BTCUSDT,Close_BTCUSDT,Volume_BTCUSDT,Quote_asset_volume_BTCUSDT,Number_of_trades_BTCUSDT,Open_ETHUSDT,High_ETHUSDT,Low_ETHUSDT,Close_ETHUSDT,Volume_ETHUSDT,Quote_asset_volume_ETHUSDT,Number_of_trades_ETHUSDT,Open_BNBUSDT,High_BNBUSDT,Low_BNBUSDT,Close_BNBUSDT,Volume_BNBUSDT,Quote_asset_volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,MACD,Signal,ADX,SlowK,SlowD,CCI,ATR,MFI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Cantidad_pos,Cantidad_neg,Cantidad_neu,Compound total,Tweets_Utilizados_coin,Cantidad_pos_coin,Cantidad_neg_coin,Cantidad_neu_coin,Compound total coin,Tweets_Utilizados_referentes,Cantidad_pos_referentes,Cantidad_neg_referentes,Cantidad_neu_referentes,Compound total referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance
902,0.120167,0.128135,0.115844,0.128164,0.243737,0.062840,0.083843,0.864346,0.915203,0.878256,0.918331,0.054236,0.178460,0.143388,0.663011,0.686122,0.660288,0.694908,0.128726,0.353230,0.503699,0.830924,0.829511,0.809577,0.820811,0.368893,0.541466,0.443672,0.138751,0.139408,0.147628,0.138751,0.127228,0.515583,0.584906,0.606897,0.298696,0.536141,0.724243,0.388226,0.211587,0.510988,0.151899,0.170213,0.352479,0.649573,1.000000,0.670051,0.629747,0.510644,0.501971,0.412587,0.618182,0.617117,0.429586,0.000000,0.000000,0.000000,0.000000,0.197248,0.139073,0.156999,0.156988,0.075976
903,0.335926,-0.088886,0.062605,0.261761,0.450071,0.001083,-0.113182,0.167239,0.548484,0.227025,0.503120,-0.172640,-0.082009,0.060163,0.327206,0.211360,0.455222,0.470235,0.181960,0.022001,0.011222,0.587176,0.370769,0.072399,0.293548,0.335355,0.194575,0.098725,0.087287,0.069804,0.215671,0.092721,0.119209,0.245114,0.149296,0.310982,0.263582,0.155436,0.424428,0.361067,0.367581,0.505913,-0.151280,0.159240,0.164609,0.142028,0.120037,-0.000675,0.356979,-0.142302,-0.296894,0.272766,0.219538,-0.175932,0.127759,-0.073758,-0.119995,0.085416,-0.241805,0.069174,-0.034834,0.457039,-0.008680,-0.200596
904,0.352774,-0.098306,0.059875,0.270484,0.464102,0.002104,-0.127585,0.151922,0.542536,0.212348,0.502046,-0.179829,-0.086356,0.055393,0.318793,0.197117,0.451881,0.476693,0.185346,0.016177,0.003161,0.600834,0.378145,0.057899,0.295052,0.334844,0.191919,0.092682,0.089111,0.066070,0.220613,0.091508,0.123531,0.227688,0.135806,0.306430,0.259597,0.134823,0.411399,0.353718,0.375387,0.504011,-0.165007,0.157825,0.160949,0.143879,0.118929,-0.007844,0.366214,-0.157814,-0.319300,0.275417,0.224856,-0.196889,0.125989,-0.082120,-0.125368,0.086098,-0.250837,0.063298,-0.047122,0.471758,-0.019156,-0.209055
905,0.362161,-0.101564,0.056326,0.275348,0.467797,0.002507,-0.136981,0.138749,0.530939,0.197806,0.493507,-0.180791,-0.088401,0.052904,0.309261,0.183703,0.445075,0.476114,0.185575,0.013120,-0.002644,0.603732,0.379818,0.047434,0.293237,0.328666,0.187758,0.087510,0.091137,0.066090,0.223871,0.090524,0.126573,0.210758,0.127073,0.299679,0.254812,0.117351,0.394703,0.343933,0.377779,0.495924,-0.172358,0.154539,0.155895,0.143201,0.116756,-0.012908,0.367325,-0.166803,-0.332458,0.274786,0.226117,-0.209260,0.122723,-0.086947,-0.127345,0.085085,-0.253379,0.059249,-0.054249,0.477419,-0.026707,-0.210641
906,0.362749,-0.095690,0.054860,0.275261,0.459248,0.000233,-0.135975,0.133966,0.518571,0.187418,0.483970,-0.174751,-0.085189,0.053731,0.303242,0.178501,0.437215,0.471181,0.184092,0.015504,-0.002872,0.597539,0.378195,0.048539,0.288066,0.318577,0.183885,0.083907,0.096139,0.070253,0.226869,0.090049,0.127322,0.199798,0.126435,0.297744,0.254078,0.108856,0.381297,0.338056,0.376983,0.487774,-0.171154,0.151156,0.150643,0.142555,0.114520,-0.014361,0.362324,-0.165915,-0.328857,0.274406,0.224457,-0.209619,0.121559,-0.083300,-0.123473,0.082646,-0.246446,0.058500,-0.055819,0.472448,-0.025250,-0.205026
907,0.356330,-0.085727,0.054379,0.270751,0.444449,-0.003345,-0.128729,0.1317

Valores predichos desnormalizados para los próximos 5 días:


,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Open_BTCUSDT,High_BTCUSDT,Low_BTCUSDT,Close_BTCUSDT,Volume_BTCUSDT,Quote_asset_volume_BTCUSDT,Number_of_trades_BTCUSDT,Open_ETHUSDT,High_ETHUSDT,Low_ETHUSDT,Close_ETHUSDT,Volume_ETHUSDT,Quote_asset_volume_ETHUSDT,Number_of_trades_ETHUSDT,Open_BNBUSDT,High_BNBUSDT,Low_BNBUSDT,Close_BNBUSDT,Volume_BNBUSDT,Quote_asset_volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,MACD,Signal,ADX,SlowK,SlowD,CCI,ATR,MFI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Cantidad_pos,Cantidad_neg,Cantidad_neu,Compound total,Tweets_Utilizados_coin,Cantidad_pos_coin,Cantidad_neg_coin,Cantidad_neu_coin,Compound total coin,Tweets_Utilizados_referentes,Cantidad_pos_referentes,Cantidad_neg_referentes,Cantidad_neu_referentes,Compound total referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance,Fecha
0,20.506784,-0.911417,6.602585,16.782532,16806502.0,4981908.00,-283530.96875,25362.621094,47831.964844,28157.009766,44605.585938,-120807.718750,-1.160862e+09,1208052.750,2242.710449,1879.748657,2626.120361,2788.064697,707720.6875,204491904.0,189026.828125,465.104523,378.469452,216.059235,331.004639,1564832.500,366927072.0,220952.781250,7.706357,6.894405,15.515876,7.946428,7.747548,38.128384,-2.793100,-1.385381,20.071831,18.387186,42.950336,-104.822586,1.579297,50.332024,-35.853439,44.905708,33339.441406,158.555176,43.093243,-0.132957,115.805305,-46.576992,-194.935989,137.016434,29.149185,-33.056988,29.597168,-1.253880,-0.479981,0.341665,-1.450829,-1.116805,-5.259930,770.864136,17.398050,-366914.68750,2024-03-18
1,21.352190,-1.395858,6.469917,17.220161,17308766.0,6430855.50,-321048.87500,24485.042969,47490.214844,27337.195312,44544.066406,-126211.867188,-1.235699e+09,1136908.250,2210.634033,1825.780518,2613.316162,2812.690430,719804.0625,175674096.0,163944.328125,471.340820,381.878448,209.518234,331.691467,1562577.125,362195392.0,210768.640625,7.786946,6.730471,15.776295,7.892811,7.905168,37.088440,-2.900338,-1.418385,19.891581,16.439400,41.797050,-109.317795,1.610285,50.164059,-39.106682,44.506588,32598.144531,160.504944,42.695572,-1.545188,118.723694,-50.853970,-211.987000,138.153763,29.734200,-37.709385,29.258238,-1.396035,-0.501472,0.344391,-1.505024,-1.168039,-7.115425,794.207642,0.982216,-382724.59375,2024-03-19
2,21.823259,-1.563462,6.297445,17.464193,17441042.0,7001646.00,-345525.40625,23730.365234,46823.832031,26524.933594,44054.832031,-126934.929688,-1.270899e+09,1099795.125,2174.289795,1774.951904,2587.235840,2810.482666,720619.5625,160548272.0,145882.015625,472.664001,382.651672,204.797562,330.862610,1535295.625,354784416.0,202052.140625,7.876434,6.731357,15.947990,7.849351,8.016109,36.078041,-2.969774,-1.467328,19.675135,14.788507,40.319073,-115.303101,1.619782,49.450172,-40.848892,43.579987,31574.496094,159.790741,41.915527,-2.542886,119.074791,-53.332527,-222.000397,137.883057,29.872919,-40.455631,28.632610,-1.478107,-0.509381,0.340339,-1.520276,-1.203353,-8.191567,803.186707,-10.849485,-385688.68750,2024-03-20
3,21.852749,-1.261326,6.226172,17.459848,17135006.0,3776779.25,-342904.50000,23456.371094,46113.140625,25944.691406,43508.492188,-122394.687500,-1.215611e+09,1112132.875,2151.340088,1755.241211,2557.111084,2791.674316,715330.1875,172347200.0,145173.171875,469.836456,381.901703,205.295898,328.502258,1490745.625,347885344.0,195980.656250,8.097415,6.914125,16.105989,7.828366,8.043427,35.423946,-2.974844,-1.481353,19.641949,13.985807,39.132378,-118.897850,1.616621,48.730705,-40.563419,42.626019,30510.750000,159.110596,41.112526,-2.829148,117.494255,-53.087776,-219.259827,137.720032,29.690226,-40.535412,28.409807,-1.416101,-0.493893,0.330586,-1.478674,-1.209881,-8.428725,795.301758,-8.567263,-375192.81250,2024-03-21
4,21.530643,-0.748956,6.202835,17.233576,16605255.0,-1298632.75,-324028.68750,23331.730469,45286.152344,25426.810547,42899.398438,-115447.250000,-1.

### Rearmado del modelo a partir de las semillas

In [58]:
with open('best_seeds.json', 'r') as f:
    best_seeds = json.load(f)

# 21 Crear y entrenar los modelos con los hiperparámetros y semillas guardados
models = []
for params_str, seed in best_seeds.items():
    params = json.loads(params_str.replace("'", "\""))
    model = KerasRegressor(build_fn=create_model, random_state=seed, **params)
    model.fit(X, y)
    models.append(model)

Epoch 1/10
28/28 [==============================] - 5s 28ms/step - loss: 32812.2305 - accuracy: 0.0252
Epoch 2/10
28/28 [==============================] - 1s 30ms/step - loss: 7.9887 - accuracy: 0.0401
Epoch 3/10
28/28 [==============================] - 1s 28ms/step - loss: 0.4735 - accuracy: 0.0160
Epoch 4/10
28/28 [==============================] - 1s 27ms/step - loss: 0.1082 - accuracy: 0.0183
Epoch 5/10
28/28 [==============================] - 1s 29ms/step - loss: 0.0868 - accuracy: 0.0355
Epoch 6/10
28/28 [==============================] - 1s 29ms/step - loss: 0.0566 - accuracy: 0.0653
Epoch 7/10
28/28 [==============================] - 1s 30ms/step - loss: 0.0473 - accuracy: 0.0607
Epoch 8/10
28/28 [==============================] - 1s 31ms/step - loss: 0.0450 - accuracy: 0.0676
Epoch 9/10
28/28 [==============================] - 1s 27ms/step - loss: 0.0445 - accuracy: 0.0687
Epoch 10/10
28/28 [==============================] - 1s 28ms/step - loss: 0.0450 - accuracy: 0.0550
Epoch

In [61]:
# 2. Predecir 5 días en el futuro con los modelos entrenados
ensemble = MultivariableVotingRegressor(models)
ensemble.fit(X, y)
future_dataset, predicted_values_desnormalized = predict_next_days(ensemble, feature_dataset, scalers, n_steps, n_features, 5)
    
display(predicted_values_desnormalized.head())

Epoch 1/10
28/28 [==============================] - 5s 30ms/step - loss: 32812.2305 - accuracy: 0.0252
Epoch 2/10
28/28 [==============================] - 1s 30ms/step - loss: 7.9887 - accuracy: 0.0401
Epoch 3/10
28/28 [==============================] - 1s 34ms/step - loss: 0.4735 - accuracy: 0.0160
Epoch 4/10
28/28 [==============================] - 1s 33ms/step - loss: 0.1082 - accuracy: 0.0183
Epoch 5/10
28/28 [==============================] - 1s 30ms/step - loss: 0.0868 - accuracy: 0.0355
Epoch 6/10
28/28 [==============================] - 1s 35ms/step - loss: 0.0566 - accuracy: 0.0653
Epoch 7/10
28/28 [==============================] - 1s 33ms/step - loss: 0.0473 - accuracy: 0.0607
Epoch 8/10
28/28 [==============================] - 1s 33ms/step - loss: 0.0450 - accuracy: 0.0676
Epoch 9/10
28/28 [==============================] - 1s 35ms/step - loss: 0.0445 - accuracy: 0.0687
Epoch 10/10
28/28 [==============================] - 1s 33ms/step - loss: 0.0450 - accuracy: 0.0550
Epoch

,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Open_BTCUSDT,High_BTCUSDT,Low_BTCUSDT,Close_BTCUSDT,Volume_BTCUSDT,Quote_asset_volume_BTCUSDT,Number_of_trades_BTCUSDT,Open_ETHUSDT,High_ETHUSDT,Low_ETHUSDT,Close_ETHUSDT,Volume_ETHUSDT,Quote_asset_volume_ETHUSDT,Number_of_trades_ETHUSDT,Open_BNBUSDT,High_BNBUSDT,Low_BNBUSDT,Close_BNBUSDT,Volume_BNBUSDT,Quote_asset_volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,MACD,Signal,ADX,SlowK,SlowD,CCI,ATR,MFI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Cantidad_pos,Cantidad_neg,Cantidad_neu,Compound total,Tweets_Utilizados_coin,Cantidad_pos_coin,Cantidad_neg_coin,Cantidad_neu_coin,Compound total coin,Tweets_Utilizados_referentes,Cantidad_pos_referentes,Cantidad_neg_referentes,Cantidad_neu_referentes,Compound total referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance,Fecha
0,20.506784,-0.911417,6.602585,16.782532,16806502.0,4981908.00,-283530.96875,25362.621094,47831.964844,28157.009766,44605.585938,-120807.718750,-1.160862e+09,1208052.750,2242.710449,1879.748657,2626.120361,2788.064697,707720.6875,204491904.0,189026.828125,465.104523,378.469452,216.059235,331.004639,1564832.500,366927072.0,220952.781250,7.706357,6.894405,15.515876,7.946428,7.747548,38.128384,-2.793100,-1.385381,20.071831,18.387186,42.950336,-104.822586,1.579297,50.332024,-35.853439,44.905708,33339.441406,158.555176,43.093243,-0.132957,115.805305,-46.576992,-194.935989,137.016434,29.149185,-33.056988,29.597168,-1.253880,-0.479981,0.341665,-1.450829,-1.116805,-5.259930,770.864136,17.398050,-366914.68750,2024-03-18
1,21.352190,-1.395858,6.469917,17.220161,17308766.0,6430855.50,-321048.87500,24485.042969,47490.214844,27337.195312,44544.066406,-126211.867188,-1.235699e+09,1136908.250,2210.634033,1825.780518,2613.316162,2812.690430,719804.0625,175674096.0,163944.328125,471.340820,381.878448,209.518234,331.691467,1562577.125,362195392.0,210768.640625,7.786946,6.730471,15.776295,7.892811,7.905168,37.088440,-2.900338,-1.418385,19.891581,16.439400,41.797050,-109.317795,1.610285,50.164059,-39.106682,44.506588,32598.144531,160.504944,42.695572,-1.545188,118.723694,-50.853970,-211.987000,138.153763,29.734200,-37.709385,29.258238,-1.396035,-0.501472,0.344391,-1.505024,-1.168039,-7.115425,794.207642,0.982216,-382724.59375,2024-03-19
2,21.823259,-1.563462,6.297445,17.464193,17441042.0,7001646.00,-345525.40625,23730.365234,46823.832031,26524.933594,44054.832031,-126934.929688,-1.270899e+09,1099795.125,2174.289795,1774.951904,2587.235840,2810.482666,720619.5625,160548272.0,145882.015625,472.664001,382.651672,204.797562,330.862610,1535295.625,354784416.0,202052.140625,7.876434,6.731357,15.947990,7.849351,8.016109,36.078041,-2.969774,-1.467328,19.675135,14.788507,40.319073,-115.303101,1.619782,49.450172,-40.848892,43.579987,31574.496094,159.790741,41.915527,-2.542886,119.074791,-53.332527,-222.000397,137.883057,29.872919,-40.455631,28.632610,-1.478107,-0.509381,0.340339,-1.520276,-1.203353,-8.191567,803.186707,-10.849485,-385688.68750,2024-03-20
3,21.852749,-1.261326,6.226172,17.459848,17135006.0,3776779.25,-342904.50000,23456.371094,46113.140625,25944.691406,43508.492188,-122394.687500,-1.215611e+09,1112132.875,2151.340088,1755.241211,2557.111084,2791.674316,715330.1875,172347200.0,145173.171875,469.836456,381.901703,205.295898,328.502258,1490745.625,347885344.0,195980.656250,8.097415,6.914125,16.105989,7.828366,8.043427,35.423946,-2.974844,-1.481353,19.641949,13.985807,39.132378,-118.897850,1.616621,48.730705,-40.563419,42.626019,30510.750000,159.110596,41.112526,-2.829148,117.494255,-53.087776,-219.259827,137.720032,29.690226,-40.535412,28.409807,-1.416101,-0.493893,0.330586,-1.478674,-1.209881,-8.428725,795.301758,-8.567263,-375192.81250,2024-03-21
4,21.530643,-0.748956,6.202835,17.233576,16605255.0,-1298632.75,-324028.68750,23331.730469,45286.152344,25426.810547,42899.398438,-115447.250000,-1.

# Light GBM Classifier